## 3.1.2. Realistic simulation setup
Script for the case of having a spot inbetween two pixels

### Plot spot with pixel diameter
- ~~create a mesh with pixel size distance corners~~
- ~~create a spot centered in the middle of the plane~~
- ~~Pass the spot through the mesh~~ 
- ~~Each pixel that has the spot goes to a micro mesh~~
- ~~Each micropixel that has the spot sum.~~
- ~~Consider the case where a spot is inbetween two pixels~~
    - Expand the mesh by 1 in each direction

### import  libraries

In [23]:
vscode = 1

In [24]:
if(vscode == 0):
    # for vscode
    # for jupyter notebook
    from mpl_toolkits.mplot3d import axes3d
    import matplotlib.pyplot as plt

    %matplotlib notebook
elif(vscode == 1):
    %matplotlib qt
    

In [25]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 
from typing import Sequence
from calib_lib import *
from matplotlib.collections import PatchCollection      # pathcoletion for multiple patches
import matplotlib.cm as cm                              # for colormaps
from matplotlib.patches import Rectangle                # for rectangle representation



DECIMALS = 2            # how many decimal places to use in print


In [26]:
F = 16                                       # focal length( in mm )
image_size = np.array([1936,1216])               # sensor size(in mm)
#image_size = np.array([11.345,7.126])
pixel_width = 5.86e-3                       # pixel size in mm
PX= image_size[0]/2.0                       # principal point x-coordinate
PY= image_size[1]/2.0                       # principal point y-coordinate
IMAGE_HEIGTH = image_size[1]
IMAGE_WIDTH = image_size[0]
THETA_X = 0                                 # roll angle
THETA_Y = 0                                 # pitch angle
THETA_Z = 0                         # yaw angle


# camera Right
THETA_X_R = 0                                 # roll angle
THETA_Y_R = 0                                 # pitch angle
THETA_Z_R= 0                                 # yaw angle
# camera Left
THETA_X_L = 0                                 # roll angle
THETA_Y_L = 0                                 # pitch angle
THETA_Z_L= 0                                 # yaw angle

C_L = np.array([0,0,0])                     # camera centre
C_R = np.array([500,0,0])

chess_dimx,chess_dimy = (13,9)
chess_sq_size = 44

## Spot between to pixels without touching the corners
- consider the centroid and mark the pixels

### Spot 

In [27]:
# Generate pixel grid
pixelGridX  = (np.arange(0,image_size[0])) * pixel_width
pixelGridY  = (np.arange(0,image_size[1])) * pixel_width
pixelGridXX, pixelGridYY = np.meshgrid(pixelGridX,pixelGridY)

In [28]:
def spot_center_coord(cx0,cy0,pixelGridX,pixelGridY):
    cx0_max = np.where((cx0 < pixelGridX))
    cx0_min = np.where((cx0 > pixelGridX))
    #print(cx0_max[0][0])
    #print(cx0_min[0][-1])
    cy0_max = np.where((cy0 < pixelGridY))
    cy0_min = np.where((cy0 > pixelGridY))
    #print(cy0_max[0][0])
    #print(cy0_min[0][-1])

    return cx0_max[0][0],cx0_min[0][-1],cy0_max[0][0],cy0_min[0][-1]

In [29]:
#spot centroid coordinates 
cx0 = 5.67248
cy0 = 3.565

In [30]:
gridSize = 1024
theta = np.linspace(0, 2*np.pi, gridSize)
r = pixel_width/4
x0 = r*np.cos(theta)+(cx0)
y0 = r*np.sin(theta)+(cy0)

circ1 = np.sqrt((pixelGridXX-cx0)**2 + (pixelGridYY-cy0)**2) <= r
circ = np.where(circ1 == True)

fig, ax = plt.subplots()
ax.plot(pixelGridXX,pixelGridYY)
ax.plot(pixelGridXX.T,pixelGridYY.T)
ax.plot(x0,y0, linewidth=4)
ax.plot(pixelGridX[circ[1][:]],pixelGridY[circ[0][:]],'r*')
ax.plot(cx0,cy0,'k.')

ax.set_xlim([cx0-(2*pixel_width),cx0+(2*pixel_width)])
ax.set_ylim([cy0-(2*pixel_width),cy0+(2*pixel_width)])

cx0max,cx0min,cy0max,cy0min = spot_center_coord(cx0,cy0,pixelGridX,pixelGridY)

# max and min points in (x,y) coordinates
cx0min_pxl = cx0min*pixel_width
cx0max_pxl = cx0max*pixel_width
cy0min_pxl = cy0min*pixel_width
cy0max_pxl = cy0max*pixel_width

#array max and min points
points_max_min = np.array([[cx0min_pxl,cy0min_pxl],
                            [cx0min_pxl,cy0max_pxl],
                            [cx0max_pxl,cy0min_pxl],
                            [cx0max_pxl,cy0max_pxl]])

# plot the max and min points
ax.plot(cx0min_pxl,cy0min_pxl,'b*')
ax.plot(cx0min_pxl,cy0max_pxl,'b*')
ax.plot(cx0max_pxl,cy0min_pxl,'b*')
ax.plot(cx0max_pxl,cy0max_pxl,'b*')

ax.set_aspect(1)
plt.show()

In [31]:
cy0min_pxl

3.56288

### Number of pixels inbetween

In [32]:
nr_pixel_x = ((cx0max*pixel_width)-(cx0min*pixel_width))/pixel_width
print("number of pixels in the x axis with the spot:",np.round(nr_pixel_x))
nr_pixel_y = ((cy0max*pixel_width)-(cy0min*pixel_width))/pixel_width
print("number of pixels in the y axis with the spot:", np.round(nr_pixel_y))

number of pixels in the x axis with the spot: 2.0
number of pixels in the y axis with the spot: 1.0


## Left pixel

In [33]:
xgrid_l = np.linspace(0,1,gridSize)*pixel_width
ygrid_l = np.linspace(0,1,gridSize)*pixel_width

x_pixel_l = xgrid_l+(cx0min*pixel_width)
y_pixel_l = ygrid_l+(cy0min*pixel_width)

xx_pixel_l, yy_pixel_l = np.meshgrid(x_pixel_l,y_pixel_l)


In [34]:
circl_total = np.sqrt((xx_pixel_l-cx0)**2 + (yy_pixel_l-cy0)**2) <= r
circl = np.where(circl_total == True)

"""
fig, ax = plt.subplots()

ax.plot(cx0,cy0,'b*')
#ax.plot(xx_one_pixel,yy_one_pixel)
#ax.plot(xx_one_pixel.T,yy_one_pixel.T)
ax.plot(x_pixel_l[circl[1][:]],y_pixel_l[circl[0][:]],'r*')

plt.show()
"""

"\nfig, ax = plt.subplots()\n\nax.plot(cx0,cy0,'b*')\n#ax.plot(xx_one_pixel,yy_one_pixel)\n#ax.plot(xx_one_pixel.T,yy_one_pixel.T)\nax.plot(x_pixel_l[circl[1][:]],y_pixel_l[circl[0][:]],'r*')\n\nplt.show()\n"

### Covered area
- Create array to store pixel percentage values,for the left and right pixels

In [35]:
# array for percentage storage
pixel_covered_percent = np.zeros((2,))

pixel_covered_left = circl[0].shape
totalpixel_area = gridSize**2
pixel_covered_percent[0] = (pixel_covered_left[0]/totalpixel_area)*100
print("pixel cover percentage: " + str(pixel_covered_percent[0]) + " %")

pixel cover percentage: 9.821605682373047 %


## Right pixel

In [36]:
xgrid_r = np.linspace(0,1,gridSize)*pixel_width
ygrid_r = np.linspace(0,1,gridSize)*pixel_width

x_pixel_r = xgrid_r+(cx0max*pixel_width) - ((nr_pixel_x/2)*(pixel_width))
y_pixel_r = ygrid_r+(cy0max*pixel_width) - ((nr_pixel_x/2)*(pixel_width))

xx_pixel_r, yy_pixel_r = np.meshgrid(x_pixel_r,y_pixel_r)

circr_total = np.sqrt((xx_pixel_r-cx0)**2 + (yy_pixel_r-cy0)**2) <= r
circr = np.where(circr_total == True)


In [37]:
"""
fig, ax = plt.subplots()

ax.plot(cx0,cy0,'b*')
#ax.plot(xx_one_pixel,yy_one_pixel)
#ax.plot(xx_one_pixel.T,yy_one_pixel.T)
ax.plot(x_pixel_r[circr[1][:]],y_pixel_r[circr[0][:]],'r*')

plt.show()
"""


"\nfig, ax = plt.subplots()\n\nax.plot(cx0,cy0,'b*')\n#ax.plot(xx_one_pixel,yy_one_pixel)\n#ax.plot(xx_one_pixel.T,yy_one_pixel.T)\nax.plot(x_pixel_r[circr[1][:]],y_pixel_r[circr[0][:]],'r*')\n\nplt.show()\n"

### Covered area

In [38]:
pixel_covered_right = circl[0].shape
totalpixel_area = gridSize**2
pixel_covered_percent[1] = (pixel_covered_right[0]/totalpixel_area)*100
print("pixel cover percentage: " + str(pixel_covered_percent[1]) + " %")

pixel cover percentage: 9.821605682373047 %


## Plot everything

In [39]:
xgrid = np.linspace(0,1,gridSize) * pixel_width
xx, yy = np.meshgrid(xgrid+(cx0min*pixel_width),xgrid+(cy0min*pixel_width))

fig, ax = plt.subplots()
ax.plot(pixelGridXX,pixelGridYY)
ax.plot(pixelGridXX.T,pixelGridYY.T)
#ax.plot(xx,yy)
#ax.plot(xx.T,yy.T)
ax.plot(x0,y0, linewidth=4)
#ax.plot(pixelGridX[circ[1][:]],pixelGridY[circ[0][:]],'r*')
#ax.plot(pixelGridX[circl[1][:]],pixelGridY[circl[0][:]],'r*')

# left pixel 
ax.plot(x_pixel_l[circl[1][:]],y_pixel_l[circl[0][:]],'r*')
# right pixel
ax.plot(x_pixel_r[circr[1][:]],y_pixel_r[circr[0][:]],'b*')

# center of the pixel
ax.plot(cx0,cy0,'k.')

ax.set_xlim([cx0-(2*pixel_width),cx0+(2*pixel_width)])
ax.set_ylim([cy0-(2*pixel_width),cy0+(2*pixel_width)])

cx0max,cx0min,cy0max,cy0min = spot_center_coord(cx0,cy0,pixelGridX,pixelGridY)

ax.plot(cx0min*pixel_width,cy0min*pixel_width,'b*')
ax.plot(cx0min*pixel_width,cy0max*pixel_width,'b*')
ax.plot(cx0max*pixel_width,cy0min*pixel_width,'b*')
ax.plot(cx0max*pixel_width,cy0max*pixel_width,'b*')

ax.set_aspect(1)
plt.show()

### Plot the pixel
- create the retangle that represents the spot

In [40]:
fig= plt.figure()
ax1= fig.add_subplot()
#axis
ax1.plot(pixelGridXX,pixelGridYY,'r')
ax1.plot(pixelGridXX.T,pixelGridYY.T,'b')
#spot
ax1.plot(x0,y0, linewidth=1)
# list with the origin values of the pixel rectangles
pixel_origin = ([[x_pixel_l[0],y_pixel_l[0]]
                ,[x_pixel_r[0],y_pixel_r[0]]])
# list for the patches
ptchs =[]
# the value 2 is hardwired, only because there is only two pixels, for now
for i in range(2):
        ptchs.append(Rectangle((pixel_origin[i][0],pixel_origin[i][1]),pixel_width,pixel_width))

p = PatchCollection(ptchs, cmap=cm.jet, alpha=0.5)
# define the color
p.set_array(pixel_covered_percent)
# set color max and min
p.set_clim([0, 100])
ax1.add_collection(p)

fig.colorbar(p)

ax1.set_xlim([cx0-(2*pixel_width),cx0+(2*pixel_width)])
ax1.set_ylim([cy0-(2*pixel_width),cy0+(2*pixel_width)])
ax1.set_aspect(1)

plt.show()

In [41]:
x_pixel = xgrid_r+(cx0max*pixel_width) - ((nr_pixel_x/2)*(pixel_width))
x_pixel 


array([5.67248   , 5.67248573, 5.67249146, ..., 5.67832854, 5.67833427,
       5.67834   ])

In [42]:
cx0maxx = points_max_min[3,0]
cy0maxx = points_max_min[3,1]

xgrid = np.linspace(0,1,gridSize)*pixel_width
ygrid = np.linspace(0,1,gridSize)*pixel_width

x_pixel = (cx0maxx)-xgrid
y_pixel = ygrid+(cy0maxx)

xx_pixel, yy_pixel = np.meshgrid(x_pixel,y_pixel)
print(x_pixel)
print(y_pixel)



[5.67834    5.67833427 5.67832854 ... 5.67249146 5.67248573 5.67248   ]
[3.56874    3.56874573 3.56875146 ... 3.57458854 3.57459427 3.5746    ]


In [43]:
def pixel_cover_finder(gridSize,cx0max,cy0max,cx0,cy0,r):
    # in order to 360 arount the point
    pixel_orient = np.array([[1,1],[1,-1],[-1,1],[-1,-1]])

    # array for percentage storage
    pixel_covered_percent = np.zeros((4,))

    xgrid = np.linspace(0,1,gridSize)*pixel_width
    ygrid = np.linspace(0,1,gridSize)*pixel_width
    for i in range(4):
        x_pixel = (cx0max)+(xgrid*pixel_orient[i,0])
        y_pixel = (cy0max)+(ygrid*pixel_orient[i,1])

        xx_pixel, yy_pixel = np.meshgrid(x_pixel,y_pixel)

        circ_total = np.sqrt((xx_pixel-cx0)**2 + (yy_pixel-cy0)**2) <= r
        circ = np.where(circ_total == True)

        pixel_covered = circ[0].shape
        totalpixel_area = gridSize**2
        pixel_covered_percent[i] = (pixel_covered[0]/totalpixel_area)*100
        #print("pixel cover percentage: " + str(pixel_covered_percent[i]) + " %")

    return pixel_covered_percent

    

In [44]:
fig,ax = plt.subplots()
pixel_orient = np.array([[1,1],[1,-1],[-1,1],[-1,-1]])
# center of the pixel
ax.plot(cx0,cy0,'k.')
#axis
ax.plot(pixelGridXX,pixelGridYY,'r')
ax.plot(pixelGridXX.T,pixelGridYY.T,'b')
#spot
ax.plot(x0,y0, linewidth=1)
# list for the patches
ptchs =[]
#pixel cover list
pixel_covered_percent = []
for j in range(len(points_max_min)):
    pixel_covered_percent.append(pixel_cover_finder(gridSize,points_max_min[j,0],points_max_min[j,1],cx0,cy0,r))
    for i in range(4):
        ptchs.append(Rectangle((points_max_min[j,0],points_max_min[j,1]), pixel_orient[i,0]*pixel_width,pixel_orient[i,1]*pixel_width))

p = PatchCollection(ptchs, cmap=cm.jet, alpha=1)
pixel_covered_percent_arr = np.array(pixel_covered_percent).flatten()
# define the color
p.set_array(pixel_covered_percent_arr)
# set color max and min
p.set_clim([0, 100])
ax.add_collection(p)

fig.colorbar(p)

ax.set_xlim([cx0-(3*pixel_width),cx0+(3*pixel_width)])
ax.set_ylim([cy0-(3*pixel_width),cy0+(3*pixel_width)])

ax.set_aspect(1)

plt.show()